In [1]:
from pathlib import Path
import glob
import os
import sys
import csv
import nltk
from collections import Counter
from nltk.tokenize import word_tokenize,wordpunct_tokenize
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures, TrigramCollocationFinder, TrigramAssocMeasures

import locale
# Set the locale to your desired setting (e.g., 'fr_FR.UTF-8' for French)
locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')

'fr_FR.UTF-8'

In [2]:
sys.path.append("/home/lucas-jerusalimiec/Documents/OCR Text/Notebooks")
from tokenizer_func  import (wordcleaner, write_words_to_file, dictionary_to_file, convert_tuple_bigrams,
convert_tuple_trigrams)

from extra_token_func import print_first_n_items, remove_keys_from_nested_dict

from additional_token_func import convert_strings_to_counts

from dict_write import write_dict_to_files_with_suffix

In [3]:
text_loc = Path("./final")
text_files = sorted(glob.glob(f"{text_loc}/*.txt"), key=locale.strxfrm)
output_folder = './tokenized/'
tokenized_folder = Path(output_folder)
tokenized_folder.mkdir(exist_ok=True)

output_unigram = f'{output_folder}unigram_counts'
unigram_folder = Path(output_unigram)
unigram_folder.mkdir(exist_ok=True)

output_bigram = f'{output_folder}bigram_counts'
bigram_folder = Path(output_bigram)
bigram_folder.mkdir(exist_ok=True)

output_trigram = f'{output_folder}trigram_counts'
trigram_folder = Path(output_trigram)
trigram_folder.mkdir(exist_ok=True)

output_collocation = f'{output_folder}collocation_counts'
collocation_folder = Path(output_collocation)
collocation_folder.mkdir(exist_ok=True)

output_trigram_collocation = f'{output_folder}trigram_collocation_counts'
trigram_collocation_folder = Path(output_trigram_collocation)
trigram_collocation_folder.mkdir(exist_ok=True)

output_underscore = f'{output_folder}underscore_bigrams'
underscore_folder = Path(output_underscore)
underscore_folder.mkdir(exist_ok=True)

output_trigram_underscore = f'{output_folder}underscore_trigrams'
trigram_underscore_folder = Path(output_trigram_underscore)
trigram_underscore_folder.mkdir(exist_ok=True)

print("Text files in the spellchecked directory:", text_files)

Text files in the spellchecked directory: ['final/0Théatre summary_corrected.txt', 'final/Théatre I.1_corrected.txt', 'final/Théatre I.2_corrected.txt', 'final/Théatre I.3_corrected.txt', 'final/Théatre I.4_corrected.txt', 'final/Théatre I.5_corrected.txt', 'final/Théatre I.6_corrected.txt', 'final/Théatre I.7_corrected.txt', 'final/Théatre I.8_corrected.txt', 'final/Théatre I.910_corrected.txt', 'final/Théatre I.911_corrected.txt', 'final/Théatre I.9_corrected.txt', 'final/Théatre II.1_corrected.txt', 'final/Théatre II.2_corrected.txt', 'final/Théatre II.3_corrected.txt', 'final/Théatre II.4_corrected.txt', 'final/Théatre II.5_corrected.txt', 'final/Théatre II.6_corrected.txt', 'final/Théatre II.7_corrected.txt', 'final/Théatre II.8_corrected.txt', 'final/Théatre II.910_corrected.txt', 'final/Théatre II.911_corrected.txt', 'final/Théatre II.9_corrected.txt', 'final/Théatre III.1_corrected.txt', 'final/Théatre III.2_corrected.txt', 'final/Théatre III.3_corrected.txt', 'final/Théatre II

In [4]:
# Open stopwords CSV file and list the contents
with open('./stop_words.csv', 'r') as f:
    stopwords = f.read().strip().split(",")
stopwords[-10:]

['dif',
 'ausquel',
 'quelz',
 'possit',
 'positum',
 'quod',
 'cóme',
 'jus',
 'dient',
 'sed']

In [5]:
tokenized_texts = {}
for txt in text_files:
    with open(txt, 'r') as f:
        content = f.read()
        file_name = txt.split('\\')[-1]
        #key = file_name.split('.')[0]
        key = os.path.splitext(os.path.basename(file_name))[0]
        tokenized_texts[key] = content
print("Raw texts:", list(tokenized_texts.keys()))       

Raw texts: ['0Théatre summary_corrected', 'Théatre I.1_corrected', 'Théatre I.2_corrected', 'Théatre I.3_corrected', 'Théatre I.4_corrected', 'Théatre I.5_corrected', 'Théatre I.6_corrected', 'Théatre I.7_corrected', 'Théatre I.8_corrected', 'Théatre I.910_corrected', 'Théatre I.911_corrected', 'Théatre I.9_corrected', 'Théatre II.1_corrected', 'Théatre II.2_corrected', 'Théatre II.3_corrected', 'Théatre II.4_corrected', 'Théatre II.5_corrected', 'Théatre II.6_corrected', 'Théatre II.7_corrected', 'Théatre II.8_corrected', 'Théatre II.910_corrected', 'Théatre II.911_corrected', 'Théatre II.9_corrected', 'Théatre III.1_corrected', 'Théatre III.2_corrected', 'Théatre III.3_corrected', 'Théatre III.4_corrected', 'Théatre III.5_corrected', 'Théatre III.6_corrected', 'Théatre III.7_corrected', 'Théatre III.8_corrected', 'Théatre III.910_corrected', 'Théatre III.911_corrected', 'Théatre III.912_corrected', 'Théatre III.913_corrected', 'Théatre III.914_corrected', 'Théatre III.915_corrected',

In [6]:
unigrams = {}

for key, value in tokenized_texts.items():
    unigram_list = wordpunct_tokenize(value)
    cleanwords = [wordcleaner(w) for w in unigram_list]
    unigrams[key] = cleanwords

for key, value in unigrams.items():
    filename = f"./tokenized/{key}.txt"
    write_words_to_file(value, filename, words_per_line=20)
    print(f"Saved content for '{key}' to {filename}")

Saved content for '0Théatre summary_corrected' to ./tokenized/0Théatre summary_corrected.txt
Saved content for 'Théatre I.1_corrected' to ./tokenized/Théatre I.1_corrected.txt
Saved content for 'Théatre I.2_corrected' to ./tokenized/Théatre I.2_corrected.txt
Saved content for 'Théatre I.3_corrected' to ./tokenized/Théatre I.3_corrected.txt
Saved content for 'Théatre I.4_corrected' to ./tokenized/Théatre I.4_corrected.txt
Saved content for 'Théatre I.5_corrected' to ./tokenized/Théatre I.5_corrected.txt
Saved content for 'Théatre I.6_corrected' to ./tokenized/Théatre I.6_corrected.txt
Saved content for 'Théatre I.7_corrected' to ./tokenized/Théatre I.7_corrected.txt
Saved content for 'Théatre I.8_corrected' to ./tokenized/Théatre I.8_corrected.txt
Saved content for 'Théatre I.910_corrected' to ./tokenized/Théatre I.910_corrected.txt
Saved content for 'Théatre I.911_corrected' to ./tokenized/Théatre I.911_corrected.txt
Saved content for 'Théatre I.9_corrected' to ./tokenized/Théatre I.9_

In [7]:
print("Unigram texts:")
for key in unigrams:
    print(key)

Unigram texts:
0Théatre summary_corrected
Théatre I.1_corrected
Théatre I.2_corrected
Théatre I.3_corrected
Théatre I.4_corrected
Théatre I.5_corrected
Théatre I.6_corrected
Théatre I.7_corrected
Théatre I.8_corrected
Théatre I.910_corrected
Théatre I.911_corrected
Théatre I.9_corrected
Théatre II.1_corrected
Théatre II.2_corrected
Théatre II.3_corrected
Théatre II.4_corrected
Théatre II.5_corrected
Théatre II.6_corrected
Théatre II.7_corrected
Théatre II.8_corrected
Théatre II.910_corrected
Théatre II.911_corrected
Théatre II.9_corrected
Théatre III.1_corrected
Théatre III.2_corrected
Théatre III.3_corrected
Théatre III.4_corrected
Théatre III.5_corrected
Théatre III.6_corrected
Théatre III.7_corrected
Théatre III.8_corrected
Théatre III.910_corrected
Théatre III.911_corrected
Théatre III.912_corrected
Théatre III.913_corrected
Théatre III.914_corrected
Théatre III.915_corrected
Théatre III.916_corrected
Théatre III.9_corrected
Théatre IV.1_corrected
Théatre IV.2_corrected
Théatre IV.

In [8]:
# Count up the tokens using a Counter() object
unigram_counts = {}
for key, value in unigrams.items():
    unigram_counts_dict = Counter(value)
    unigram_counts[key] = unigram_counts_dict

print("Unigram Counts:")
for key in unigram_counts:
    print(key)

print_first_n_items(unigram_counts, 25)

Unigram Counts:
0Théatre summary_corrected
Théatre I.1_corrected
Théatre I.2_corrected
Théatre I.3_corrected
Théatre I.4_corrected
Théatre I.5_corrected
Théatre I.6_corrected
Théatre I.7_corrected
Théatre I.8_corrected
Théatre I.910_corrected
Théatre I.911_corrected
Théatre I.9_corrected
Théatre II.1_corrected
Théatre II.2_corrected
Théatre II.3_corrected
Théatre II.4_corrected
Théatre II.5_corrected
Théatre II.6_corrected
Théatre II.7_corrected
Théatre II.8_corrected
Théatre II.910_corrected
Théatre II.911_corrected
Théatre II.9_corrected
Théatre III.1_corrected
Théatre III.2_corrected
Théatre III.3_corrected
Théatre III.4_corrected
Théatre III.5_corrected
Théatre III.6_corrected
Théatre III.7_corrected
Théatre III.8_corrected
Théatre III.910_corrected
Théatre III.911_corrected
Théatre III.912_corrected
Théatre III.913_corrected
Théatre III.914_corrected
Théatre III.915_corrected
Théatre III.916_corrected
Théatre III.9_corrected
Théatre IV.1_corrected
Théatre IV.2_corrected
Théatre IV

In [9]:
# Remove specified keys from the dictionary
stripped_unigrams = remove_keys_from_nested_dict(unigram_counts, stopwords)

print_first_n_items(stripped_unigrams, 30)

First 30 items in 0Théatre summary_corrected:
: 439
nature: 17
choses: 12
causes: 9
ordre: 8
matiere: 7
chose: 6
mouvement: 5
hypostase: 5
forme: 5
doctrine: 4
naturelles: 4
generation: 4
plantes: 4
fans: 4
vie: 4
hystoire: 4
telle: 3
faire: 3
femble: 3
doit: 3
ailleurs: 3
queftions: 3
cieux: 3
animaux: 3
preceder: 3
doétrine: 3
air: 3
celestes: 3
physicien: 3

First 30 items in Théatre I.1_corrected:
: 578
corps: 17
chose: 15
forme: 14
naturel: 12
fin: 8
cause: 8
choses: 6
nature: 6
definition: 6
aucune: 6
efficiente: 6
ariftote: 5
donc: 5
fçauoir: 4
bien: 4
fcience: 4
fans: 4
cela: 4
fubie: 3
phyfique: 3
principes: 3
premiere: 3
quelle: 3
fciences: 3
pen: 2
ve: 2
premier: 2
moyen: 2
causes: 2

First 30 items in Théatre I.2_corrected:
: 550
monde: 17
cause: 7
dieu: 7
forme: 5
choses: 5
cela: 5
premiere: 4
fuft: 4
admirable: 4
pu: 4
temps: 4
felon: 4
donc: 3
faire: 3
chose: 3
infinie: 3
corps: 3
eternel: 3
là: 3
ii: 3
beaucoup: 3
routes: 3
entre: 3
nature: 3
efté: 3
fin: 3
efgal: 3
pet

In [10]:
dictionary_to_file(stripped_unigrams, unigram_folder, 'unigram_counts')

Saved 0Théatre summary_corrected_unigram_counts.csv in tokenized/unigram_counts
Saved Théatre I.1_corrected_unigram_counts.csv in tokenized/unigram_counts
Saved Théatre I.2_corrected_unigram_counts.csv in tokenized/unigram_counts
Saved Théatre I.3_corrected_unigram_counts.csv in tokenized/unigram_counts
Saved Théatre I.4_corrected_unigram_counts.csv in tokenized/unigram_counts
Saved Théatre I.5_corrected_unigram_counts.csv in tokenized/unigram_counts
Saved Théatre I.6_corrected_unigram_counts.csv in tokenized/unigram_counts
Saved Théatre I.7_corrected_unigram_counts.csv in tokenized/unigram_counts
Saved Théatre I.8_corrected_unigram_counts.csv in tokenized/unigram_counts
Saved Théatre I.910_corrected_unigram_counts.csv in tokenized/unigram_counts
Saved Théatre I.911_corrected_unigram_counts.csv in tokenized/unigram_counts
Saved Théatre I.9_corrected_unigram_counts.csv in tokenized/unigram_counts
Saved Théatre II.1_corrected_unigram_counts.csv in tokenized/unigram_counts
Saved Théatre I

In [11]:
bigrams = {}

for key, value in unigrams.items():
    unigram_list = [word for word in value if word.lower() not in stopwords]
    #unigram_list = [word for word in value]
    bigrams_list = list(nltk.bigrams(unigram_list))
    bigrams[key] = bigrams_list

print("Bigrams:")
for key in bigrams:
    print(key)

Bigrams:
0Théatre summary_corrected
Théatre I.1_corrected
Théatre I.2_corrected
Théatre I.3_corrected
Théatre I.4_corrected
Théatre I.5_corrected
Théatre I.6_corrected
Théatre I.7_corrected
Théatre I.8_corrected
Théatre I.910_corrected
Théatre I.911_corrected
Théatre I.9_corrected
Théatre II.1_corrected
Théatre II.2_corrected
Théatre II.3_corrected
Théatre II.4_corrected
Théatre II.5_corrected
Théatre II.6_corrected
Théatre II.7_corrected
Théatre II.8_corrected
Théatre II.910_corrected
Théatre II.911_corrected
Théatre II.9_corrected
Théatre III.1_corrected
Théatre III.2_corrected
Théatre III.3_corrected
Théatre III.4_corrected
Théatre III.5_corrected
Théatre III.6_corrected
Théatre III.7_corrected
Théatre III.8_corrected
Théatre III.910_corrected
Théatre III.911_corrected
Théatre III.912_corrected
Théatre III.913_corrected
Théatre III.914_corrected
Théatre III.915_corrected
Théatre III.916_corrected
Théatre III.9_corrected
Théatre IV.1_corrected
Théatre IV.2_corrected
Théatre IV.3_corr

In [12]:
bigram_counts = {}

for key, value in bigrams.items():
    string_bigrams = convert_tuple_bigrams(value)
    bigramCount = convert_strings_to_counts(string_bigrams)
    bigram_counts[key] = bigramCount

print("Bigram Counts:")
for key in bigram_counts:
    print(key)

print_first_n_items(bigram_counts, 30)

Bigram Counts:
0Théatre summary_corrected
Théatre I.1_corrected
Théatre I.2_corrected
Théatre I.3_corrected
Théatre I.4_corrected
Théatre I.5_corrected
Théatre I.6_corrected
Théatre I.7_corrected
Théatre I.8_corrected
Théatre I.910_corrected
Théatre I.911_corrected
Théatre I.9_corrected
Théatre II.1_corrected
Théatre II.2_corrected
Théatre II.3_corrected
Théatre II.4_corrected
Théatre II.5_corrected
Théatre II.6_corrected
Théatre II.7_corrected
Théatre II.8_corrected
Théatre II.910_corrected
Théatre II.911_corrected
Théatre II.9_corrected
Théatre III.1_corrected
Théatre III.2_corrected
Théatre III.3_corrected
Théatre III.4_corrected
Théatre III.5_corrected
Théatre III.6_corrected
Théatre III.7_corrected
Théatre III.8_corrected
Théatre III.910_corrected
Théatre III.911_corrected
Théatre III.912_corrected
Théatre III.913_corrected
Théatre III.914_corrected
Théatre III.915_corrected
Théatre III.916_corrected
Théatre III.9_corrected
Théatre IV.1_corrected
Théatre IV.2_corrected
Théatre IV.

In [13]:
dictionary_to_file(bigram_counts, bigram_folder, 'bigram_counts')

Saved 0Théatre summary_corrected_bigram_counts.csv in tokenized/bigram_counts
Saved Théatre I.1_corrected_bigram_counts.csv in tokenized/bigram_counts
Saved Théatre I.2_corrected_bigram_counts.csv in tokenized/bigram_counts
Saved Théatre I.3_corrected_bigram_counts.csv in tokenized/bigram_counts
Saved Théatre I.4_corrected_bigram_counts.csv in tokenized/bigram_counts
Saved Théatre I.5_corrected_bigram_counts.csv in tokenized/bigram_counts
Saved Théatre I.6_corrected_bigram_counts.csv in tokenized/bigram_counts
Saved Théatre I.7_corrected_bigram_counts.csv in tokenized/bigram_counts
Saved Théatre I.8_corrected_bigram_counts.csv in tokenized/bigram_counts
Saved Théatre I.910_corrected_bigram_counts.csv in tokenized/bigram_counts
Saved Théatre I.911_corrected_bigram_counts.csv in tokenized/bigram_counts
Saved Théatre I.9_corrected_bigram_counts.csv in tokenized/bigram_counts
Saved Théatre II.1_corrected_bigram_counts.csv in tokenized/bigram_counts
Saved Théatre II.2_corrected_bigram_count

In [14]:
trigrams = {}

for key, value in unigrams.items():
    unigram_list = [word for word in value if word.lower() not in stopwords]
    #unigram_list = [word for word in value]
    trigrams_list = list(nltk.trigrams(unigram_list))
    trigrams[key] = trigrams_list

print("Trigrams:")
for key in bigrams:
    print(key)

Trigrams:
0Théatre summary_corrected
Théatre I.1_corrected
Théatre I.2_corrected
Théatre I.3_corrected
Théatre I.4_corrected
Théatre I.5_corrected
Théatre I.6_corrected
Théatre I.7_corrected
Théatre I.8_corrected
Théatre I.910_corrected
Théatre I.911_corrected
Théatre I.9_corrected
Théatre II.1_corrected
Théatre II.2_corrected
Théatre II.3_corrected
Théatre II.4_corrected
Théatre II.5_corrected
Théatre II.6_corrected
Théatre II.7_corrected
Théatre II.8_corrected
Théatre II.910_corrected
Théatre II.911_corrected
Théatre II.9_corrected
Théatre III.1_corrected
Théatre III.2_corrected
Théatre III.3_corrected
Théatre III.4_corrected
Théatre III.5_corrected
Théatre III.6_corrected
Théatre III.7_corrected
Théatre III.8_corrected
Théatre III.910_corrected
Théatre III.911_corrected
Théatre III.912_corrected
Théatre III.913_corrected
Théatre III.914_corrected
Théatre III.915_corrected
Théatre III.916_corrected
Théatre III.9_corrected
Théatre IV.1_corrected
Théatre IV.2_corrected
Théatre IV.3_cor

In [15]:
trigram_counts = {}

for key, value in trigrams.items():
    string_trigrams = convert_tuple_trigrams(value)
    trigramCount = convert_strings_to_counts(string_trigrams)
    trigram_counts[key] = trigramCount

print("Trigram Counts:")
for key in trigram_counts:
    print(key)
    
print_first_n_items(trigram_counts, 30)

Trigram Counts:
0Théatre summary_corrected
Théatre I.1_corrected
Théatre I.2_corrected
Théatre I.3_corrected
Théatre I.4_corrected
Théatre I.5_corrected
Théatre I.6_corrected
Théatre I.7_corrected
Théatre I.8_corrected
Théatre I.910_corrected
Théatre I.911_corrected
Théatre I.9_corrected
Théatre II.1_corrected
Théatre II.2_corrected
Théatre II.3_corrected
Théatre II.4_corrected
Théatre II.5_corrected
Théatre II.6_corrected
Théatre II.7_corrected
Théatre II.8_corrected
Théatre II.910_corrected
Théatre II.911_corrected
Théatre II.9_corrected
Théatre III.1_corrected
Théatre III.2_corrected
Théatre III.3_corrected
Théatre III.4_corrected
Théatre III.5_corrected
Théatre III.6_corrected
Théatre III.7_corrected
Théatre III.8_corrected
Théatre III.910_corrected
Théatre III.911_corrected
Théatre III.912_corrected
Théatre III.913_corrected
Théatre III.914_corrected
Théatre III.915_corrected
Théatre III.916_corrected
Théatre III.9_corrected
Théatre IV.1_corrected
Théatre IV.2_corrected
Théatre IV

In [16]:
dictionary_to_file(trigram_counts, trigram_folder, 'trigram_counts')

Saved 0Théatre summary_corrected_trigram_counts.csv in tokenized/trigram_counts
Saved Théatre I.1_corrected_trigram_counts.csv in tokenized/trigram_counts
Saved Théatre I.2_corrected_trigram_counts.csv in tokenized/trigram_counts
Saved Théatre I.3_corrected_trigram_counts.csv in tokenized/trigram_counts
Saved Théatre I.4_corrected_trigram_counts.csv in tokenized/trigram_counts
Saved Théatre I.5_corrected_trigram_counts.csv in tokenized/trigram_counts
Saved Théatre I.6_corrected_trigram_counts.csv in tokenized/trigram_counts
Saved Théatre I.7_corrected_trigram_counts.csv in tokenized/trigram_counts
Saved Théatre I.8_corrected_trigram_counts.csv in tokenized/trigram_counts
Saved Théatre I.910_corrected_trigram_counts.csv in tokenized/trigram_counts
Saved Théatre I.911_corrected_trigram_counts.csv in tokenized/trigram_counts
Saved Théatre I.9_corrected_trigram_counts.csv in tokenized/trigram_counts
Saved Théatre II.1_corrected_trigram_counts.csv in tokenized/trigram_counts
Saved Théatre I

In [17]:
colloc_dict = {}
colloc_counts = {}

for key, value in unigrams.items():
    unigram_list = [word for word in value if word.lower() not in stopwords]
    bigram_finder = BigramCollocationFinder.from_words(unigram_list)
    bigram_finder.apply_freq_filter(3)  # Make sure all collocations have occurred at least 5 times
    collocations = bigram_finder.nbest(BigramAssocMeasures.pmi, 500)
    filtered_collocations = [bigram for bigram in collocations if '' not in bigram]
    colloc_dict[key] = filtered_collocations
    
    # Initialize Counter for colloc_counts
    bigram_count_dict = Counter()

    # Count the occurrences of each bigram in the text
    bigram_finder = BigramCollocationFinder.from_words(unigram_list)
    bigram_freqs = bigram_finder.ngram_fd.items()
    
    # Filter bigram counts based on collocations
    for bigram, count in bigram_freqs:
        if bigram in filtered_collocations:
            bigram_count_dict[bigram] = count

    colloc_counts[key] = bigram_count_dict

print("Collocations:")
for key, value in colloc_dict.items():
    print(key)
    # for w1, w2 in value:
    #     print(' ', w1, w2)

print("Collocation Counts:")
for key in colloc_counts:
    print(key)
    # Print first n items, assuming print_first_n_items function is defined elsewhere
    for item, count in colloc_counts[key].most_common(30):
        bigram = " ".join(item)
        print(f"{bigram} {count}")
    print()

dictionary_to_file(colloc_counts, collocation_folder, 'collocation_counts')

Collocations:
0Théatre summary_corrected
Théatre I.1_corrected
Théatre I.2_corrected
Théatre I.3_corrected
Théatre I.4_corrected
Théatre I.5_corrected
Théatre I.6_corrected
Théatre I.7_corrected
Théatre I.8_corrected
Théatre I.910_corrected
Théatre I.911_corrected
Théatre I.9_corrected
Théatre II.1_corrected
Théatre II.2_corrected
Théatre II.3_corrected
Théatre II.4_corrected
Théatre II.5_corrected
Théatre II.6_corrected
Théatre II.7_corrected
Théatre II.8_corrected
Théatre II.910_corrected
Théatre II.911_corrected
Théatre II.9_corrected
Théatre III.1_corrected
Théatre III.2_corrected
Théatre III.3_corrected
Théatre III.4_corrected
Théatre III.5_corrected
Théatre III.6_corrected
Théatre III.7_corrected
Théatre III.8_corrected
Théatre III.910_corrected
Théatre III.911_corrected
Théatre III.912_corrected
Théatre III.913_corrected
Théatre III.914_corrected
Théatre III.915_corrected
Théatre III.916_corrected
Théatre III.9_corrected
Théatre IV.1_corrected
Théatre IV.2_corrected
Théatre IV.3

In [18]:
trigram_colloc_dict = {}
trigram_colloc_counts = {}

for key, value in unigrams.items():
    unigram_list = [word for word in value if word.lower() not in stopwords]
    trigram_finder = TrigramCollocationFinder.from_words(unigram_list)
    trigram_finder.apply_freq_filter(3)  # Ensure all collocations have occurred at least 5 times
    collocations = trigram_finder.nbest(TrigramAssocMeasures.pmi, 500)
    filtered_collocations = [trigram for trigram in collocations if '' not in trigram]
    trigram_colloc_dict[key] = filtered_collocations
    
    # Initialize Counter for trigram_colloc_counts
    trigram_count_dict = Counter()

    # Count the occurrences of each trigram in the text
    trigram_freqs = trigram_finder.ngram_fd.items()
    
    # Filter trigram counts based on collocations
    for trigram, count in trigram_freqs:
        if trigram in filtered_collocations:
            trigram_count_dict[trigram] = count

    trigram_colloc_counts[key] = trigram_count_dict

print("Trigram Collocations:")
for key, value in trigram_colloc_dict.items():
    print(key)
    #for w1, w2, w3 in value:
    #    print(' ', w1, w2, w3)

print("Trigram Collocation Counts:")
for key in trigram_colloc_counts:
    print(key)
    # Print first n items, assuming print_first_n_items function is defined elsewhere
    for item, count in trigram_colloc_counts[key].most_common(30):
        trigram = " ".join(item)
        print(f"{trigram} {count}")
    print()

dictionary_to_file(trigram_colloc_counts, trigram_collocation_folder, 'trigram_collocation_counts')

Trigram Collocations:
0Théatre summary_corrected
Théatre I.1_corrected
Théatre I.2_corrected
Théatre I.3_corrected
Théatre I.4_corrected
Théatre I.5_corrected
Théatre I.6_corrected
Théatre I.7_corrected
Théatre I.8_corrected
Théatre I.910_corrected
Théatre I.911_corrected
Théatre I.9_corrected
Théatre II.1_corrected
Théatre II.2_corrected
Théatre II.3_corrected
Théatre II.4_corrected
Théatre II.5_corrected
Théatre II.6_corrected
Théatre II.7_corrected
Théatre II.8_corrected
Théatre II.910_corrected
Théatre II.911_corrected
Théatre II.9_corrected
Théatre III.1_corrected
Théatre III.2_corrected
Théatre III.3_corrected
Théatre III.4_corrected
Théatre III.5_corrected
Théatre III.6_corrected
Théatre III.7_corrected
Théatre III.8_corrected
Théatre III.910_corrected
Théatre III.911_corrected
Théatre III.912_corrected
Théatre III.913_corrected
Théatre III.914_corrected
Théatre III.915_corrected
Théatre III.916_corrected
Théatre III.9_corrected
Théatre IV.1_corrected
Théatre IV.2_corrected
Théa

In [19]:
underscore_dict = {}
for key, value in unigrams.items():

    tokenized_words = unigrams.get(key)
    collocations = colloc_dict.get(key)
    
    colloc_words = []
    
    # Iterate through the words making new versions combining collocations
    i = 0
    while i < len(tokenized_words) - 1:
        # If we find a collocation, add and advance by two words
        if (tokenized_words[i], tokenized_words[i + 1]) in collocations:
            colloc_words.append('_'.join((tokenized_words[i], tokenized_words[i + 1])))
            i += 2
        # Otherwise, advance by one word
        else:
            colloc_words.append(tokenized_words[i])
            i += 1

    # Add the last word (if any)
    if i == len(tokenized_words) - 1:
        colloc_words.append(tokenized_words[i])
    underscore_dict[key] = colloc_words

print("Underscore Dictionary:")
for key in underscore_dict:
    print(key)

write_dict_to_files_with_suffix(underscore_dict, underscore_folder, 'underscore_bigrams')

Underscore Dictionary:
0Théatre summary_corrected
Théatre I.1_corrected
Théatre I.2_corrected
Théatre I.3_corrected
Théatre I.4_corrected
Théatre I.5_corrected
Théatre I.6_corrected
Théatre I.7_corrected
Théatre I.8_corrected
Théatre I.910_corrected
Théatre I.911_corrected
Théatre I.9_corrected
Théatre II.1_corrected
Théatre II.2_corrected
Théatre II.3_corrected
Théatre II.4_corrected
Théatre II.5_corrected
Théatre II.6_corrected
Théatre II.7_corrected
Théatre II.8_corrected
Théatre II.910_corrected
Théatre II.911_corrected
Théatre II.9_corrected
Théatre III.1_corrected
Théatre III.2_corrected
Théatre III.3_corrected
Théatre III.4_corrected
Théatre III.5_corrected
Théatre III.6_corrected
Théatre III.7_corrected
Théatre III.8_corrected
Théatre III.910_corrected
Théatre III.911_corrected
Théatre III.912_corrected
Théatre III.913_corrected
Théatre III.914_corrected
Théatre III.915_corrected
Théatre III.916_corrected
Théatre III.9_corrected
Théatre IV.1_corrected
Théatre IV.2_corrected
Thé

In [20]:
trigram_underscore_dict = {}

for key, tokenized_words in unigrams.items():
    collocations = trigram_colloc_dict.get(key, [])
    colloc_words = []
    i = 0
    while i < len(tokenized_words) - 2:
        # If we find a trigram collocation, add and advance by three words
        trigram = (tokenized_words[i], tokenized_words[i + 1], tokenized_words[i + 2])
        if trigram in collocations:
            colloc_words.append('_'.join(trigram))
            i += 3
        else:
            colloc_words.append(tokenized_words[i])
            i += 1
    # Add the last words (if any)
    while i < len(tokenized_words):
        colloc_words.append(tokenized_words[i])
        i += 1
    trigram_underscore_dict[key] = colloc_words

print("Trigram underscore Dictionary:")
for key in trigram_underscore_dict:
    print(key)

write_dict_to_files_with_suffix(trigram_underscore_dict, trigram_underscore_folder, 'underscore_trigrams')

Trigram underscore Dictionary:
0Théatre summary_corrected
Théatre I.1_corrected
Théatre I.2_corrected
Théatre I.3_corrected
Théatre I.4_corrected
Théatre I.5_corrected
Théatre I.6_corrected
Théatre I.7_corrected
Théatre I.8_corrected
Théatre I.910_corrected
Théatre I.911_corrected
Théatre I.9_corrected
Théatre II.1_corrected
Théatre II.2_corrected
Théatre II.3_corrected
Théatre II.4_corrected
Théatre II.5_corrected
Théatre II.6_corrected
Théatre II.7_corrected
Théatre II.8_corrected
Théatre II.910_corrected
Théatre II.911_corrected
Théatre II.9_corrected
Théatre III.1_corrected
Théatre III.2_corrected
Théatre III.3_corrected
Théatre III.4_corrected
Théatre III.5_corrected
Théatre III.6_corrected
Théatre III.7_corrected
Théatre III.8_corrected
Théatre III.910_corrected
Théatre III.911_corrected
Théatre III.912_corrected
Théatre III.913_corrected
Théatre III.914_corrected
Théatre III.915_corrected
Théatre III.916_corrected
Théatre III.9_corrected
Théatre IV.1_corrected
Théatre IV.2_corre